# Lab 4: Random Forest

In this Lab, you will implement a very simplified version of the Random Forest classifier. Make sure that you check the videos of lecture 4 before starting this Lab:
- Decision Tree and Random Forest: https://youtu.be/YSy9S2OsXNo

Random Forst (RF) works as follows:

- Given a dataset $X \in \mathbb{R}^{n \times d}$, RF selects randomly some features (i.e. `n_features`) from the dataset (where `n_features << d`). It also selects a random subset of the data (with `n_samples` data-points). Then, it builds a Decision Tree from those selected features and samples.

- Repeats this process `n_trees` times so that you have a number of `n_trees` Decision Trees built from different random combinations of features and different random subsets of data.

- To predict, RF takes each of the `n_trees` built Decision Trees and predict the outputs (classes); then it calculates the votes for each predicted class-label and takes the mode (most frequent label). In other words, considers the high voted predicted label as the final prediction from the random forest algorithm.

## Loading the dataset:
The code below will load a training dataset into variables `X` and `y` as usual.

In [178]:
%matplotlib inline
import numpy as np

filename = "datasets/university-admission-dataset.csv"
mydata = np.genfromtxt(filename, delimiter=",")
X = mydata[:, :2]
y = mydata[:, -1]

""" TODO:
Print a small subset of X and y to see how the data looks like.
"""
print(X[:5])
print(y[:5])

[[34.62365962 78.02469282]
 [30.28671077 43.89499752]
 [35.84740877 72.90219803]
 [60.18259939 86.3085521 ]
 [79.03273605 75.34437644]]
[0. 0. 0. 1. 1.]


## Using sklearn.tree.DecisionTreeClassifier
To simplify the implementation of our Random Forest classifier, we will use an existing implementation of the `DecisionTreeClassifier` available in the sklearn library. Read the following code to see an example of how to use the `DecisionTreeClassifier`.

In [179]:
""" TODO:
Read and run the following example code to see how to use a 
simple DecisionTreeClassifier to make predictions.
"""

from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0).fit(X, y) # Training
preds = clf.predict(X) # Predicting

print("Predictions:", preds)

acc = np.mean(preds == y) * 100
print("Training Accuracy: {}%".format(acc))

Predictions: [0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0.
 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1.]
Training Accuracy: 100.0%


## Implementing a simplified random forest classifier
Complete the code below to implement a random forest classifier.

In Python, to select a list of `k` random integers between `0` and `nbr` (excluded), you can use `ids = np.random.choice(nbr, k, replace=False)`. The keyword `replace=False` means that we don't want to select the same number more than once (so, ids with contain `k` unique integers).

Also, to compute the mode (most common or frequent label), you can make use of the function `scipy.stats.mode` if you want. Read its documentation here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mode.html


In [180]:
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import PolynomialFeatures



""" TODO:
Complete the definition of the function train(...) which allows to train an ensemble 
of decision tree classifiers. Each trained decision tree clf is saved together with 
its corresponding features indices (used to train clf) into the list clfs.

Arguments:
*** X: the training dataset.
*** y: the training labels corresponding to X.
*** n_features: the number of features to select randomly to build a decision tree.
*** n_samples: the number of instances to select randomly to build a decision tree.
*** n_trees: the number of decision trees to build.
"""
def train(X, y, n_features, n_samples, n_trees):
    n, d = X.shape # n: number of rows, and d: number of columns
    clfs = [] # used to store the decision trees with their corresponding features indices.
    
    for itr in range(n_trees):
        pass
        # TODO: select randomly n_features indices between 0 and d and store them in ids_columns
        ids_columns = np.random.choice(d, n_features, replace=False) # you can use np.random.choice here
        
        
        # TODO: select randomly n_samples indices between 0 and n and store them in ids_samples
        ids_samples = np.random.choice(n, n_samples, replace=False) # you can use np.random.choice here
        
        
        # TODO: select the subset of X corresponding to the rows ids_samples and the columns ids_columns
        Xsub = X[ids_samples, ids_columns]
        Xsub = Xsub.reshape(-1,1)
        
        # TODO: select the subset of y corresponding to ids_samples
        ysub = y[ids_samples]
        # if (itr == 0):
        #     print(ysub.shape)
        #     print(Xsub.shape)
        
        # TODO: train a decision tree classifier clf, based on the training subset: Xsub, ysub
        # clf = DecisionTreeClassifier(criterion="gini", splitter="best", max_depth=5, min_samples_split=3)
        clf = DecisionTreeClassifier().fit(Xsub, ysub)
            
        # TODO: save the trained decision tree classifier (clf) and the corresponding 
        # feature indices (ids_columns) used to train this classifier, into the list clfs.
        clfs.append((clf, ids_columns))
        
    return clfs


""" TODO:
Complete the definition of the function predict(clfs, X) which allows to predict the class-labels 
for the instances in the given dataset X, using the ensemble clfs returned by your function previous function.
"""
def predict(clfs, X):
    predictions = []
    for x in X:
        sub_predictions = []
        for c in clfs:
            target_feature = c[1][0]
            feature_value = x[target_feature].reshape(-1,1)
            sub_pred = c[0].predict(feature_value)
            sub_predictions.append(sub_pred)
        pred =stats.mode(sub_predictions)[0][0][0]
        predictions.append(pred)
    return predictions


""" TODO:
Uncomment the code below to test your functions:
"""
clfs = train(X, y, n_features=1, n_samples=50, n_trees=10)
# print(clfs)
y_pred = predict(clfs, X)
# print(y_pred)
acc = np.mean(y_pred == y) * 100
print("Training Accuracy: {}%".format(acc))

Training Accuracy: 95.0%


## Optional: Testing your implementation a dataset with multiple features
This part is optional.

Test you random forest implementation on a dataset with more features (e.g. you can add more polynomial features to the previous dataset or use any other dataset with a high number of features).

Compute the generalization accuracy of your random forest on that dataset using a 10-fold-cross-validation.

In [181]:
# TODO (optional) ...
poly = PolynomialFeatures(degree=4)
X_new = poly.fit_transform(X)
# print(X_new[:5])
clfs = train(X_new, y, n_features=1, n_samples=50, n_trees=10)
y_pred = predict(clfs, X_new)
acc = np.mean(y_pred == y) * 100
print("Training Accuracy: {}%".format(acc))


Training Accuracy: 97.0%
